# `03-station-schedule-seperation.ipynb`
02 에서 만든 데이터로 쪼게기.

In [1]:
%load_ext autoreload
%autoreload 2

import warnings

warnings.simplefilter(action="ignore", category=FutureWarning)

In [2]:
import pandas as pd
from tqdm import tqdm
import util
import stationUtil

In [3]:
DAYTYPE = ["토요일", "일요일", "공휴일", "평일"]
PATH_STATION_SCHEDULE_REFINE_DAYTYPE = [
    f"../trimmed_data/stationSchedule{day}.json" for day in DAYTYPE
]
PATH_STATION_INTERVAL_DAYTYPE = [
    f"../trimmed_data/stationSchedule{day}-SEPERATED.json" for day in DAYTYPE
]

## 1. 데이터 불러오기

In [14]:
path = util.PATH_STATION_INFO_REFINE
sName_sCode = pd.read_json(path).iloc[:, [0, 1]].transpose().to_dict()
tmp={}
for val in sName_sCode.values():
    lNumber = stationUtil.get_line(val["StationCode"])
    station_name = val["StationName"]
    station_name = station_name.replace("·", ".")
    station_name = station_name.replace("국제금융센터", "국제금융")
    tmp[(lNumber,station_name)] = val["StationCode"]
sName_sCode = tmp
util.savePickle(sName_sCode, "../trimmed_data/stationNameCode.pickel")

########## 환승역은 중복되므로 에러가 발생함.
# sName_sCode = {}
# for key, val in pd.read_json(path).iloc[:, [0, 1]].transpose().to_dict().items():
#     # for k,v in key.items():
#     sName_sCode[val["StationName"]] = val["StationCode"]

# readme.md 참조.
# sName_sCode = {key.replace("·", "."): val for key, val in sName_sCode.items()}
# sName_sCode = {
#     key.replace("국제금융센터", "국제금융"): val for key, val in sName_sCode.items()
# }
print(sName_sCode[(1, "서면")], sName_sCode[(2, "서면")])

119 219


In [15]:
stationNameCode = util.loadPickle("../trimmed_data/stationNameCode.pickel")

for day, path, savepath in zip(
    DAYTYPE, PATH_STATION_SCHEDULE_REFINE_DAYTYPE, PATH_STATION_INTERVAL_DAYTYPE
):
    db = pd.read_json(path, encoding="UTF-8")
    column = db.columns.to_list()

    df = pd.DataFrame({c: [] for c in column})
    for i in tqdm(db.index):
        tCode, lNumber, dVect, sList, aSchedule, dSchedule = db.loc[i, :]
        # print(lNumber,sList);import sys;sys.exit()
        for s, a, d in zip(sList, aSchedule, dSchedule):
            _unit = pd.DataFrame(
                {
                    column[0]: [int(tCode)],
                    column[2]: [int(dVect)],
                    column[1]: [int(lNumber)],
                    column[3]: [int( stationNameCode[(lNumber,s)] )],
                    column[4]: [a],
                    column[5]: [d],
                    "DayType": day,
                }
            )
            # import sys;sys.exit()
            # df.loc[len(df)] = pd.Series({'A': 6, 'B': 'f'})   # 1 minute 28.3 sec
            # df = df._append(_unit)                            # 38.5 sec
            df = pd.concat([df, _unit], ignore_index=True)  # 35.6 sec

    # 모든 숫자는 정수형 입니다. 타입 수정을 위해 수정합니다.
    trim_target = column[:8]
    for idx in trim_target:
        df[idx] = pd.to_numeric(df[idx], downcast="integer")

    with open(savepath, mode="w", encoding="UTF-8") as f:
        f.write(df.to_json(index=False))
        print(f"SAVE >>> {savepath}")

100%|██████████| 1277/1277 [00:39<00:00, 32.50it/s]


SAVE >>> ../trimmed_data/stationSchedule토요일-SEPERATED.json


100%|██████████| 278/278 [00:03<00:00, 81.98it/s]


SAVE >>> ../trimmed_data/stationSchedule일요일-SEPERATED.json


100%|██████████| 924/924 [00:36<00:00, 25.56it/s]


SAVE >>> ../trimmed_data/stationSchedule공휴일-SEPERATED.json


100%|██████████| 1354/1354 [00:51<00:00, 26.10it/s]

SAVE >>> ../trimmed_data/stationSchedule평일-SEPERATED.json


In [ ]:
df.tail()

,TrainCode,LineNumber,DirectionVector,StationList,ArriveSchedule,DepartSchedule,DayType
38446,4310,4,7,410,1476,1476.0,평일
38447,4310,4,7,411,1478,1479.0,평일
38448,4310,4,7,412,1481,1481.0,평일
38449,4310,4,7,413,1482,1483.0,평일
38450,4310,4,7,414,1484,NaN,평일
